## Analytics on Glassdoor Reviews and Yelp Category Data

### PSTAT 135/235: Big Data Analytics
### In-class activity

### OBJECTIVE  
#### In this notebook, you will perform some basic analytics on review and category data.
#### This will entail performing operations on *RDDs*, and using *list comprehensions*.
#### Read in the dataset and perform the steps requested below.

### Config Setup

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .getOrCreate()

sc = spark.sparkContext

Read in the dataset  
The dataset is saved in the *data* folder. Notice the pathing below, with NO forward slash in front of *data*

In [4]:
# df = sc.textFile("gs://##BUCKET-NAME##/path/to/file")
df = sc.textFile("reviews_and_categories.csv")

In [5]:
df

reviews_and_categories.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
df.take(3)

['index,review_emp_txt,categories',
 '0,,"[\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"',
 '1,,[]']

In [7]:
header = df.first()

In [8]:
header

'index,review_emp_txt,categories'

look at some data

In [9]:
df.take(5)

['index,review_emp_txt,categories',
 '0,,"[\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"',
 '1,,[]',
 '2,,"[\'other\', \'food & beverages\']"',
 '3,"Some franchise owners dock hours. Pros Good discounts on the food. Cons The location where I was working, in North Fresno near Riverpark Mall, was ran by the owner s father who treated the female staff with contempt and derision. Would yell at the staff, in front of the guests, if they didn t exactly follow his formula for making the sandwiches (even when the staff were trying to fulfill the special requests of the guests). He would clock out the closers (with or without their knowledge) before they were done with their tasks, and ask employees to stay an hour or two past the end of their shift, but would not pay them for their time.","[\'lunch\', \'best sandwich\', \'entertainment\', \'restaurants\', \'sub\', \'arizona\', \'quick\', \'social networks\', \'washington\', \'catering reno\', \'establishment

### Filter rows with matching substring

In [10]:
myfunc = lambda r: "jukebox" in r
df.filter(myfunc).take(10)

['35,,"[\'jukebox\', \'sports bar\', \'sports bars\', \'credit cards\', \'music\', \'beer\', \'no outdoor seating\', \'american (traditional)\', \'price\', \'drinks\', \'point of interest\', \'establishment\', \'restaurant\', \'mexican\', \'food\']"']

### Get non-header records  
notice the categories are in a list. we split on the beginning of the list to separate cols (1,2) from (3)  
then we split again to obtain (1,2,3)

In [16]:
data = df.filter(lambda r: r != header) \
        .map(lambda row: (row.split('[')[0], row.split('[')[1:])) \
        .map(lambda x: (x[0].split(',')[0], ''.join(x[0].split(',')[1:]), x[1]))

### Print Raw data row

Let's dissect the multiline code by executing line by line

In [11]:
df.filter(lambda r: r != header).take(2)

['0,,"[\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"',
 '1,,[]']

### Split (1,2) from (3)

In [12]:
df.filter(lambda r: r != header) \
    .map(lambda row: (row.split('[')[0], row.split('[')[1:])).take(2)

[('0,,"',
  ['\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"']),
 ('1,,', [']'])]

### Split into (1,2,3)

Notice the quote and ']' characters we will clean these up later

In [21]:
df.filter(lambda r: r != header) \
    .map(lambda row: (row.split('[')[0], row.split('[')[1:])) \
    .map(lambda x: (x[0].split(',')[0], ''.join(x[0].split(',')[1:]), x[1])) \
    .take(4)

[('0',
  '"',
  ['\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"']),
 ('1', '', [']']),
 ('2', '"', ['\'other\', \'food & beverages\']"']),
 ('3',
  '"Some franchise owners dock hours. Pros Good discounts on the food. Cons The location where I was working in North Fresno near Riverpark Mall was ran by the owner s father who treated the female staff with contempt and derision. Would yell at the staff in front of the guests if they didn t exactly follow his formula for making the sandwiches (even when the staff were trying to fulfill the special requests of the guests). He would clock out the closers (with or without their knowledge) before they were done with their tasks and ask employees to stay an hour or two past the end of their shift but would not pay them for their time.""',
  ['\'lunch\', \'best sandwich\', \'entertainment\', \'restaurants\', \'sub\', \'arizona\', \'quick\', \'social networks\', \'washington\', \'catering reno\', \'establishment

### Compare to Raw data

In [20]:
df.take(5)

['index,review_emp_txt,categories',
 '0,,"[\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"',
 '1,,[]',
 '2,,"[\'other\', \'food & beverages\']"',
 '3,"Some franchise owners dock hours. Pros Good discounts on the food. Cons The location where I was working, in North Fresno near Riverpark Mall, was ran by the owner s father who treated the female staff with contempt and derision. Would yell at the staff, in front of the guests, if they didn t exactly follow his formula for making the sandwiches (even when the staff were trying to fulfill the special requests of the guests). He would clock out the closers (with or without their knowledge) before they were done with their tasks, and ask employees to stay an hour or two past the end of their shift, but would not pay them for their time.","[\'lunch\', \'best sandwich\', \'entertainment\', \'restaurants\', \'sub\', \'arizona\', \'quick\', \'social networks\', \'washington\', \'catering reno\', \'establishment

### Print review comments only

In [22]:
data.map(lambda x: x[1]).take(5)

['"',
 '',
 '"',
 '"Some franchise owners dock hours. Pros Good discounts on the food. Cons The location where I was working in North Fresno near Riverpark Mall was ran by the owner s father who treated the female staff with contempt and derision. Would yell at the staff in front of the guests if they didn t exactly follow his formula for making the sandwiches (even when the staff were trying to fulfill the special requests of the guests). He would clock out the closers (with or without their knowledge) before they were done with their tasks and ask employees to stay an hour or two past the end of their shift but would not pay them for their time.""',
 '"']

### Print categories only

In [173]:
data.map(lambda x: x[2]).take(5)

[['\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"'],
 [']'],
 ['\'other\', \'food & beverages\']"'],
 ['\'lunch\', \'best sandwich\', \'entertainment\', \'restaurants\', \'sub\', \'arizona\', \'quick\', \'social networks\', \'washington\', \'catering reno\', \'establishment\', \'nevada\', \'restaurant\', \'wraps\', \'qsr\', \'small business\', \'meal takeaway\', \'hospitality\', \'sandwich\', \'franchise\', \'seminars\', \'deli\', \'point of interest\', \'sandwiches\', \'port\', \'other\', \'food\', \'party trays reno\', \'service\', \'entrepeneur\', \'franchises\', \'fast food\', \'grillers\', \'griller\', \'salad\', \'management\', \'businesses\', \'self employed\', \'wrap\', \'submarine\', \'delis\', \'lake tahoe\', \'boss\', \'salads\', \'trade shows\', \'eating places\', \'franchising\', \'reno\', \'subs\', \'phoenix\']"'],
 ['\'french\', \'event space\', \'wine, full bar & cocktails\', \'credit cards\', \'drinks\', \'price\', \'outdoor seating\'

**Get the record count**

In [184]:
data.count()

1305

### Records with reviews only

store records with non-empty *review_emp_txt*

In [23]:
# filter out records with '"' (from rows w category data) and '' (from rows w null category data)
data_has_review = data.filter(lambda r: r[1] != '"') \
                      .filter(lambda r: r[1] != '')
data_has_review.count()

305

In [24]:
data_has_review.take(3)

[('3',
  '"Some franchise owners dock hours. Pros Good discounts on the food. Cons The location where I was working in North Fresno near Riverpark Mall was ran by the owner s father who treated the female staff with contempt and derision. Would yell at the staff in front of the guests if they didn t exactly follow his formula for making the sandwiches (even when the staff were trying to fulfill the special requests of the guests). He would clock out the closers (with or without their knowledge) before they were done with their tasks and ask employees to stay an hour or two past the end of their shift but would not pay them for their time.""',
  ['\'lunch\', \'best sandwich\', \'entertainment\', \'restaurants\', \'sub\', \'arizona\', \'quick\', \'social networks\', \'washington\', \'catering reno\', \'establishment\', \'nevada\', \'restaurant\', \'wraps\', \'qsr\', \'small business\', \'meal takeaway\', \'hospitality\', \'sandwich\', \'franchise\', \'seminars\', \'deli\', \'point of int

### Return the count of records where review contains the word *awesome*

In [25]:
awesome_records = data_has_review.filter(lambda r: 'awesome' in r[1])

In [30]:
awesome_records.count()

10

Print the records where review contains the word *awesome*. What does `collect` do? ([Documentation](https://spark.apache.org/docs/latest/api/python/reference/index.html))

In [29]:
awesome_records.collect()

[('280',
  '"Manager Pros Great environment awesome owners! I was happy to come to work every day and face any challenges presented. It was a pleasant environment with a lot of opportunity to advance if you worked hard. Cons I had performed the role of a manager long before I was given the raise to match. There is a high turn over rate and it s hard to find good team members who want to work hard.""',
  ['\'entertainment\', \'credit cards\', \'restaurants\', \'green smoothie\', \'colleges and universities\', \'smoothies and juice bars\', \'menus\', \'food, beverages & tobacco\', \'1\', \'juice bar\', \'establishment\', \'las vegas\', \'meal takeaway\', \'price\', \'juice bars & smoothies\', \'hospitality\', \'tallahassee\', \'point of interest\', \'shopping\', \'sandwiches\', \'health foods\', \'wheat grass\', \'other\', \'food\', \'smoothies\', \'hampton university\', \'franchises\', \'$$\', \'restaurant chains\', \'blimey limey\', \'food and beverages\', \'dinner, lunch & more\', \'e

### Lowercase all reviews, then return the count of records where review contains the word *awesome*

In [197]:
awesome_records_lower = data_has_review.map(lambda r: (r[0], r[1].lower(), r[2])) \
                .filter(lambda r: 'awesome' in r[1]) \

In [198]:
awesome_records_lower.collect()

[('280',
  '"manager pros great environment awesome owners! i was happy to come to work every day and face any challenges presented. it was a pleasant environment with a lot of opportunity to advance if you worked hard. cons i had performed the role of a manager long before i was given the raise to match. there is a high turn over rate and it s hard to find good team members who want to work hard.""',
  ['\'entertainment\', \'credit cards\', \'restaurants\', \'green smoothie\', \'colleges and universities\', \'smoothies and juice bars\', \'menus\', \'food, beverages & tobacco\', \'1\', \'juice bar\', \'establishment\', \'las vegas\', \'meal takeaway\', \'price\', \'juice bars & smoothies\', \'hospitality\', \'tallahassee\', \'point of interest\', \'shopping\', \'sandwiches\', \'health foods\', \'wheat grass\', \'other\', \'food\', \'smoothies\', \'hampton university\', \'franchises\', \'$$\', \'restaurant chains\', \'blimey limey\', \'food and beverages\', \'dinner, lunch & more\', \'e

In [200]:
awesome_records_lower.count()

12

### Return the top 10 most frequent categories

Preprocess the categories by:  
* stripping characters: &nbsp; [ &nbsp; ] &nbsp;  ' &nbsp;  "  
* trim spaces before and after words  
* lowercase
* removing blank categories

NOTE: Be sure to keep terms together, for example 'jet skiing' should not become 'jet', 'skiing'

In [42]:
cats=data.map(lambda r: r[2]) 

In [43]:
cats.take(3)

[['\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"'],
 [']'],
 ['\'other\', \'food & beverages\']"']]

In [44]:
cats_flat = cats.map(lambda row: [token.replace('[','') \
                                .replace(']','') \
                                .replace('\\','') \
                                .replace("'",'') \
                                .replace('"','') \
                                .strip() \
                                .lower() for token in row]) \
                                .flatMap(lambda x: x) \
                                .filter(lambda x: x != '') \
                                .flatMap(lambda x: x.split(',')) \
                                .map(lambda x: (x,1)) \
                                .reduceByKey(lambda x,y:x+y) \
                                .map(lambda x:(x[1],x[0])) \
                                .sortByKey(False) 

In [45]:
cats_flat.take(10)

[(705, ' establishment'),
 (701, ' food'),
 (671, ' point of interest'),
 (643, ' restaurant'),
 (473, ' price'),
 (471, ' other'),
 (311, ' menus'),
 (254, ' eating places'),
 (253, ' dining options'),
 (232, ' credit cards')]

In [38]:
cats.map(lambda row: [token.replace('[','') \
    .replace(']','') \
    .replace('\\','') \
    .replace("'",'') \
    .replace('"','') \
    .strip() \
    .lower() for token in row]) \
    .flatMap(lambda x: x) \
    .filter(lambda x: x != '') \
    .flatMap(lambda x: x.split(',')) \
    .map(lambda x: (x,1)) \
    .reduceByKey(lambda x,y:x+y) \
    .map(lambda x:(x[1],x[0])) \
    .sortByKey(False) 

PythonRDD[31] at RDD at PythonRDD.scala:53

### Starting point

In [41]:
cats.take(5)

[['\'point of interest\', \'mexican\', \'establishment\', \'food\', \'restaurant\']"'],
 [']'],
 ['\'other\', \'food & beverages\']"'],
 ['\'lunch\', \'best sandwich\', \'entertainment\', \'restaurants\', \'sub\', \'arizona\', \'quick\', \'social networks\', \'washington\', \'catering reno\', \'establishment\', \'nevada\', \'restaurant\', \'wraps\', \'qsr\', \'small business\', \'meal takeaway\', \'hospitality\', \'sandwich\', \'franchise\', \'seminars\', \'deli\', \'point of interest\', \'sandwiches\', \'port\', \'other\', \'food\', \'party trays reno\', \'service\', \'entrepeneur\', \'franchises\', \'fast food\', \'grillers\', \'griller\', \'salad\', \'management\', \'businesses\', \'self employed\', \'wrap\', \'submarine\', \'delis\', \'lake tahoe\', \'boss\', \'salads\', \'trade shows\', \'eating places\', \'franchising\', \'reno\', \'subs\', \'phoenix\']"'],
 ['\'french\', \'event space\', \'wine, full bar & cocktails\', \'credit cards\', \'drinks\', \'price\', \'outdoor seating\'

### Clean up characters

In [46]:
cats.map(lambda row: [
    token.replace('[','') \
        .replace(']','') \
        .replace('\\','') \
        .replace("'",'') \
        .replace('"','') \
        .strip() \
        .lower() for token in row
    ]).take(5)

[['point of interest, mexican, establishment, food, restaurant'],
 [''],
 ['other, food & beverages'],
 ['lunch, best sandwich, entertainment, restaurants, sub, arizona, quick, social networks, washington, catering reno, establishment, nevada, restaurant, wraps, qsr, small business, meal takeaway, hospitality, sandwich, franchise, seminars, deli, point of interest, sandwiches, port, other, food, party trays reno, service, entrepeneur, franchises, fast food, grillers, griller, salad, management, businesses, self employed, wrap, submarine, delis, lake tahoe, boss, salads, trade shows, eating places, franchising, reno, subs, phoenix'],
 ['french, event space, wine, full bar & cocktails, credit cards, drinks, price, outdoor seating, $$, point of interest, eating places, reservations, restaurant, establishment, cocktail bars, wine bar, bar, other, food, french restaurant']]

### flatMap

In [50]:
cats.map(lambda row: [
    token.replace('[','') \
        .replace(']','') \
        .replace('\\','') \
        .replace("'",'') \
        .replace('"','') \
        .strip() \
        .lower() for token in row
    ]) \
    .flatMap(lambda x: x) \
    .take(5)

['point of interest, mexican, establishment, food, restaurant',
 '',
 'other, food & beverages',
 'lunch, best sandwich, entertainment, restaurants, sub, arizona, quick, social networks, washington, catering reno, establishment, nevada, restaurant, wraps, qsr, small business, meal takeaway, hospitality, sandwich, franchise, seminars, deli, point of interest, sandwiches, port, other, food, party trays reno, service, entrepeneur, franchises, fast food, grillers, griller, salad, management, businesses, self employed, wrap, submarine, delis, lake tahoe, boss, salads, trade shows, eating places, franchising, reno, subs, phoenix',
 'french, event space, wine, full bar & cocktails, credit cards, drinks, price, outdoor seating, $$, point of interest, eating places, reservations, restaurant, establishment, cocktail bars, wine bar, bar, other, food, french restaurant']

### Make tuple and reduceByKey

[Reduce by key documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.reduceByKey.html?highlight=reducebykey#pyspark.RDD.reduceByKey)

In [54]:
cats.map(lambda row: [
    token.replace('[','') \
        .replace(']','') \
        .replace('\\','') \
        .replace("'",'') \
        .replace('"','') \
        .strip() \
        .lower() for token in row
    ]) \
    .flatMap(lambda x: x) \
    .filter(lambda x: x != '') \
    .flatMap(lambda x: x.split(',')) \
    .map(lambda x: (x,1)) \
    .reduceByKey(lambda x,y:x+y) \
    .take(5)



[('point of interest', 47),
 (' mexican', 95),
 (' establishment', 705),
 (' food', 701),
 (' restaurant', 643)]

In [55]:
cats.map(lambda row: [
    token.replace('[','') \
        .replace(']','') \
        .replace('\\','') \
        .replace("'",'') \
        .replace('"','') \
        .strip() \
        .lower() for token in row
    ]) \
    .flatMap(lambda x: x) \
    .filter(lambda x: x != '') \
    .flatMap(lambda x: x.split(',')) \
    .map(lambda x: (x,1)) \
    .reduceByKey(lambda x,y:x+y) \
    .map(lambda x:(x[1],x[0])) \
    .sortByKey(False) \
    .take(5)


[(705, ' establishment'),
 (701, ' food'),
 (671, ' point of interest'),
 (643, ' restaurant'),
 (473, ' price')]